L’exercice proposé ci-dessous est basé sur un problème de recalage de modèle à partir de mesures de rigidités transversales pour des enveloppes poids lourd. Les données sont mises à disposition dans les fichiers « 315 80 R 22.5.txt, 385 55 R 22.5.txt, 495 45 R 22.5.txt » avec 3 colonnes : 
- Charge (noté $Fz$) en Newton,
- Pression (noté $P$) en Bar,
- Rigidité (noté $Dz$) en N/°. 

Les coefficients à optimiser sont notés $a1, a2, a3, a4$ et $a5$. Le modèle à ajuster est le suivant :

$ \displaystyle Dz = (a1 \times P + a2) \times \sin \left(a3 \times \arctan{\left(\frac{Fz}{a4 \times P + a5}\right)} \right)$

Cet exercice est séparé en dex parties. La première consiste à résoudre le problème d'identification de paramètres en utilisant des outils classiques d'optimisation. La seconde vise à trouver les solutions via une approche d'optimisation bayesienne.

## Identification par Optimisation Classique

### Chargement des Données

Veuillez tout d'abord charger les mesures de rigidités de dérive contenues dans le fichier " 315 80 R 22.5.txt".

In [1]:
import numpy as np
import scipy as sp
from scipy import linalg
from scipy.optimize import leastsq as LB
import matplotlib.pyplot as plt
 
%matplotlib notebook

chemin = "315 80 R 22.5.txt"
mat = np.loadtxt(chemin)
z = mat[:,0]   # N
p  = mat[:,1]   # Bar
dz = mat[:,2]   # N/°

Ces mesures correspondents à des valeurs de rigidités de dérive pour trois niveaux de pression 7, 8.5 et 9.5 bars. Chaque palier de pression comporte 7 points de mesure.
Afficher les mesures $Dz(Fz)$ pour chaque palier de pression

In [2]:
# Affichage des Mesures

z1=z[:7]
z2=z[7:14]
z3=z[14:21]

p1=sorted(p[:7])
p2=sorted(p[7:14])
p3=sorted(p[14:21])

dz1=sorted(dz[:7])
dz2=sorted(dz[7:14])
dz3=sorted(dz[14:21])

# affichage de Dz(Fz)

In [40]:
plt.plot(z1,dz1)
plt.savefig("dz1(fz1)")

<IPython.core.display.Javascript object>

In [41]:
plt.plot(z2,dz2)
plt.savefig("dz2(fz2)")

<IPython.core.display.Javascript object>

In [42]:
plt.plot(z3,dz3)
plt.savefig("dz3(fz3)")

<IPython.core.display.Javascript object>

In [35]:
p1

[6.98, 6.98, 6.99, 6.99, 6.99, 7.0, 7.01]

In [34]:
plt.scatter(p1,z1)
plt.savefig("ex1_1")

<IPython.core.display.Javascript object>

In [4]:
plt.scatter(p2,z2)
plt.savefig("ex1_2")

<IPython.core.display.Javascript object>

In [5]:
plt.scatter(p3,z3)
plt.savefig("ex1_3")

<IPython.core.display.Javascript object>

In [6]:
print(z1)
print(p1)
print(dz1)

[11736. 25447. 39179. 48986. 58788. 68591. 78373.]
[6.98, 6.98, 6.99, 6.99, 6.99, 7.0, 7.01]
[1557.64, 3237.7, 4438.22, 4961.65, 5287.09, 5448.14, 5460.77]


In [7]:
print(z1)
print(z2)
print(z3)

[11736. 25447. 39179. 48986. 58788. 68591. 78373.]
[11730. 25454. 39178. 49003. 58799. 68603. 78363.]
[11734. 25463. 39198. 49004. 58790. 68606. 78402.]


In [8]:
print(p1)
p4=[p1[0],p1[2],p1[5],p1[6]]
z4=[z1[0],z1[2],z1[5],z1[6]]
dz4=[dz1[0],dz1[2],dz1[5],dz1[6]]
print(p4)
print(z4)

[6.98, 6.98, 6.99, 6.99, 6.99, 7.0, 7.01]
[6.98, 6.99, 7.0, 7.01]
[11736.0, 39179.0, 68591.0, 78373.0]


In [9]:
plt.scatter(p4,z4)
plt.savefig("ex1_4")

<IPython.core.display.Javascript object>

### Estimation des Valeurs Initiales

L’exercice est un problème d’optimisation continue, non linéaire et sans contrainte. Il est donc nécessaire d'initialiser les paramètres afin de pouvoir lancer un algorithme d'optimisation type recherche linéaire ou région de confiance.

Comment, à votre avis, peut-on essayer d’estimer les coefficients du modèle à partir des mesures ? Pour vous aider, n’oubliez pas que la fonction sinus est bornée. De plus, cadeau, je vous conseille d’approcher a3 par la valeur 2. Il vous reste seulement 4 petits coef à trouver…

Second conseil, le modèle peut également s’écrire sous la forme suivante, idéale quand on connaît déjà $a1, a2$ et $a3$, avec quelques restrictions cependant sur le domaine de définition des fonctions $1/x$, $\arctan$ et $\arcsin$:

$ \displaystyle a4 \times P + a5 = \frac{Fz}{\tan \left(\frac{1}{3} \times \arcsin\left(\frac{Dz}{a1 \times P + a2} \right) \right)}$

# rechere a1 et a2

In [10]:
a1=(z4[0]-z4[1])/(p4[0]-p4[1])
a1

2744300.0000000587

In [11]:
a2=z4[0]-a1*p4[0]
a2

-19143478.00000041

# recherche a4 a5

In [12]:
def f(P,Fz,Dz):
    arg_arcsin = Dz / (a1 * P + a2)

    # Calcul de l'arc sinus
    arcsin_result = np.arcsin(arg_arcsin)

    # Calcul de l'argument de la tangente
    arg_tan = (1 / 3) * arcsin_result

    # Calcul de la tangente
    tan_result = np.tan(arg_tan)

    # Calcul final de l'expression
    result = Fz / tan_result
    return result
    #return fz/(np.tan((1/3)*np.arcsin(dz/(a1*p+a2))))

In [13]:
data=f(p1[0],z1[0],dz1)
data45=[]
for i in range(len(p4)):
    data45.append(f(p4[i],z4[i],dz4[i]))

In [14]:
data45

[264317.4551483314, 1034852.6245760171, 2512839.3452871474, 4047281.2302307207]

In [15]:
plt.scatter(p4,data45)
plt.savefig("ex1_5")

<IPython.core.display.Javascript object>

In [16]:
a4=(data45[0]-data45[1])/(p4[0]-p4[1])
a4

77053516.94277021

In [17]:
a5=data45[0]-a4*p4[0]
a5

-537569230.8053877

In [18]:
# Estimation initiale de a1 et a2
#a1=2744300.0000000587
#a2=-19143478.00000041
# Estimation initiale de a4 et a5    
#a4=77053516.94277021
#a5=-537569230.8053877
a3=2

Superposer les courbes de mesures et de prédictions obtenues avec l'initialisation trouvée. Que constatez-vous?

In [19]:
# Fonction permettant de simuler le modèle
# Entrées: paramètres du modèle
# Return: la valeur de Dz 

def fonction_a_ajuster(parametres, P, Dz, Fz):
    a1, a2, a3, a4, a5 = parametres
    return (a1 * P + a2) * np.sin(a3 * np.arctan(Fz / (a4 * P + a5))) - Dz

In [20]:
# Affichage de la superposition mesures/prédictions



### Fine Tuning des valeurs

Dans le but d'affiner les valeurs obtenues à l'issue de la phase d'initialisation, il est nécessaire de faire appel à un algorithme d'optimisation non linéaire.

En l'occurence, il s'agit d'un problème de curve fitting. 

Quel critère est-il pertinent de considérer? 

Quel type d'algorithme peut-on mettre en oeuvre? 

Implémenter l'optimisation des valeurs des coefficients.

In [36]:
def cost_function(params):
    a1, a2, a3, a4, a5 = params
    total_cost = 0
    
    for dataset in data:
        Fz, P, Dz = dataset[:, 0], dataset[:, 1], dataset[:, 2]
        predicted_Dz = (a1 * P + a2) * np.sin(a3 * np.arctan(Fz / (a4 * P + a5)))
        total_cost += np.sum((predicted_Dz - Dz)**2)
    
    return total_cost

In [21]:
# Fonction permettant de calculer la fonction coût
# Entrées: paramètres à optimiser
# Return: la valeur du critère à optimiser 

parametres_initiaux=[a1,a2,a3,a4,a5]
P_data = np.array(p1)
Dz_data = np.array(dz1)
Fz_data = np.array(z1)

In [37]:
# Optimisation des paramètres
parametres_optimaux, _ = LB(cost_function, parametres_initiaux, args=(P_data, Dz_data, Fz_data))


TypeError: cost_function() takes 1 positional argument but 4 were given

Superposer les courbes de mesures et de prédictions obtenues avec les valeurs finales trouvées. A-t-on améliorer la qualité de prédiction du modèle par rapport à la phase d'initialisation? Comment peut-on le quantifier?

In [23]:
# Affichage de la superposition mesures/prédictions avec les nouvelles valeurs optimisées des paramètres
print("Paramètres optimaux :", parametres_optimaux)

# Générer des valeurs ajustées pour le tracé
Dz_ajustee = fonction_a_ajuster(parametres_optimaux, P_data, Dz_data, Fz_data) + Dz_data

# Tracer les données et la courbe ajustée
plt.scatter(P_data, Dz_data, label="Données réelles")
plt.plot(P_data, Dz_ajustee, label="Courbe ajustée", color='red')
plt.legend()
plt.savefig("ex2_1")
plt.show()

Paramètres optimaux : [-6.01997377e+05  1.00014957e+07  2.65863330e-02  3.41077759e+07
 -2.36852107e+08]


<IPython.core.display.Javascript object>

## Identification par Optimisation Bayesienne

Comme précisé plus haut, cette partie vise à résoudre le problème d'identification via une approche bayesienne. Pour ce faire, nous allons nous appuyer sur le package python scikit-optimize (https://scikit-optimize.github.io/stable/).

In [26]:
from skopt import gp_minimize
from skopt.plots import plot_gaussian_process, plot_convergence

### Premier Essai

Nous travaillons toujours avec la même fonction objectif utilisée dans la première partie du TP. On va considérer de plus les bornes suivantes:
- $200 \leq a1 \leq 300$
- $3000 \leq a2 \leq 4500$
- $1 \leq a3 \leq 3$
- $1e4 \leq a4 \leq 2e4$
- $-5e3 \leq a5 \leq -2e3$

La fonction d'acquisition qui sera utilisée dans un premier temps est l'amélioration espérée (Expected Improvement). Nous fixerons le nombre d'appels à la fonction coût originale à 100. Enfin, on utilise un plan de type LHS avec 50 points pour construire le modèle de krigeage avant la permière minimisation de la fonction d'acquisition.

A l'aide de la documentation du package scikit-optimize, implémenter cette première tentative d'optimisation bayesienne et visualiser le graphe de convergence. A-t-on le même niveau de convergence qu'observé dans la première section du TP? Comment pourrait-on améliorer la convergence?

In [27]:
# Fonction permettant de calculer la fonction coût au format attendu par scikit-optimize
# Entrées: paramètres à optimiser
# Return: la valeur du critère à optimiser 



In [28]:
# Définition des bornes


In [29]:
# Estimation des paramètres par optimisation bayesienne



In [30]:
# Affichage de l'évolution de la fonction coût au fil des itérations via la fonction "plot_convergence" de scikit-optimize



### Amélioration de la configuration de l'algorithme

Essayer différents nombres de points d'évaluation et les fonctions d'acquisition PI et LCB pour tenter d'améliorer le paramétrage de l'algorithme. A partir de l'analyse des graphiques de convergence et de fonctions d'acquisition, quelle est la configuration la plus efficiente?

In [31]:
# Tests et évaluation avec différentes options du solver



